In [4]:
from PIL import Image
import numpy as np
import skimage
import skimage.morphology
import skimage.feature
import skan
from scipy import ndimage as ndi
from tifffile import imsave
from scipy.interpolate import interp1d

def pruning(skeleton_image,skeleton_obj,distance_transform):
    n_path=-1
    while n_path!=0:
        summary=skan.summarize(skeleton_obj)
        """
        path_num=[i for i in summary.index if (summary['branch-type'][i] == 2 or summary['branch-type'][i] == 3 or 
                                            (summary['branch-type'][i] == 1 and
                                            (summary['image-coord-dst-0'][i] > 472 or summary['image-coord-dst-0'][i] < 40 or 
                                            summary['image-coord-dst-1'][i] > 472 or summary['image-coord-dst-1'][i] < 40 or 
                                            summary['image-coord-dst-2'][i] >126 or summary['image-coord-dst-2'][i] < 10 or
                                            summary['branch-distance'][i] < 17)))]
        """
        path_num=[i for i in summary.index if (summary['branch-type'][i] == 0 or summary['branch-type'][i] == 3 or (summary['branch-type'][i] == 1 and
                (distance_transform[int(summary['image-coord-dst-0'][i]),int(summary['image-coord-dst-1'][i]),int(summary['image-coord-dst-2'][i])]*2>summary['euclidean-distance'][i] or
                distance_transform[int(summary['image-coord-src-0'][i]),int(summary['image-coord-src-1'][i]),int(summary['image-coord-src-2'][i])]*2>summary['euclidean-distance'][i]) ))]
        if(len(path_num)==n_path):
            print("break")
            print(n_path)
            break
        else:
            n_path=len(path_num)
        
        coordinates = np.array((skeleton_obj.coordinates),dtype=int)
        for i in path_num:
            path=skeleton_obj.path(i)
            for j in range(len(path)):
                skeleton_image[coordinates[path[j],0],coordinates[path[j],1],coordinates[path[j],2]] = False
        skeleton_obj = skan.Skeleton(skeleton_image)


def soma_prunning(shape,skeleton_obj,distance_transform):
    n_path = 0
    dleta_n_path = -1
    skeleton_image = np.zeros(shape,dtype=bool)
    while dleta_n_path!=0:
        summary=skan.summarize(skeleton_obj)
        dleta_n_path = n_path - len(summary.index)
        print(len(summary.index))
        n_path = len(summary.index)
        if(len(summary.index)<=1 or dleta_n_path==0):
            #np.save('np arrays/soma_skeleton', skeleton_image)
            return skeleton_image
        coordinates = np.array((skeleton_obj.coordinates),dtype=int)

        skeleton_image = np.zeros(np.shape(skeleton_image),dtype=bool)
        nodes = np.zeros(len(skeleton_obj.coordinates))
        paths = skeleton_obj.paths_list()
        for i in range(len(paths)):
            nodes[paths[i][0]] = True
            nodes[paths[i][-1]] = True
        nodes_index = np.where(nodes==True)[0]
        for node in nodes_index:
            branch=[]
            type2_branches=0
            euclidean_dist=[]
            for i in summary.index:
                if(paths[i][0]==node or paths[i][-1]==node):
                    if(summary['branch-type'][i]==2):
                        type2_branches = type2_branches + 1
                    elif(summary['branch-type'][i]==1):
                        branch.append(i)
                        euclidean_dist.append(summary['euclidean-distance'][i])
            if(len(branch)<2 or type2_branches>1):
                continue
            i_max = np.argmax(euclidean_dist)
            path = paths[branch[i_max]]
            for j in range(len(path)):
                skeleton_image[coordinates[path[j],0],coordinates[path[j],1],coordinates[path[j],2]] = True  
            if(type2_branches==0):
                branch.pop(i_max)
                euclidean_dist.pop(i_max)
                i_max = np.argmax(euclidean_dist)
                path = paths[branch[i_max]]
                for j in range(len(path)):
                    skeleton_image[coordinates[path[j],0],coordinates[path[j],1],coordinates[path[j],2]] = True
        for i in summary.index:
            if(summary['branch-type'][i]==2):
                if(paths[i][0]==paths[i][-1]):
                    print('cycle found in soma skeleton')
                    return skeleton_image
                for j in range(len(paths[i])):
                    skeleton_image[coordinates[paths[i][j],0],coordinates[paths[i][j],1],coordinates[paths[i][j],2]] = True
                
        skeleton_obj = skan.Skeleton(skeleton_image)


In [2]:
distance_transform = ndi.distance_transform_edt(np.load('np arrays/foreground.npy'))
soma_skeleton_image = np.load('np arrays/soma_skeleton.npy') #dendrites
skeleton_obj = skan.Skeleton(soma_skeleton_image)

soma_image = np.zeros([3,np.shape(soma_skeleton_image)[0],np.shape(soma_skeleton_image)[1],np.shape(soma_skeleton_image)[2]])
soma_image[0,:,:,:] = soma_skeleton_image
soma_image[1,:,:,:] = soma_prunning(np.shape(soma_skeleton_image),skeleton_obj,distance_transform)
soma_image[2,:,:,:] = np.load('np arrays/soma_mask.npy')
imsave('images/soma_skeleton.tif',np.einsum('ijkl->lijk', soma_image))


25
5
1


In [ ]:
from mayavi import mlab
for i in range(100):
    mlab.plot3d(theta,y,z,tube_radius = diam[i])
mlab.show()

In [3]:
soma_skeleton = skan.Skeleton(soma_image[1,:,:,:])
soma_path = soma_skeleton.path(0)
coordinates = soma_skeleton.coordinates
soma_skeleton_coordinates = np.array([coordinates[soma_path,0],coordinates[soma_path,1],coordinates[soma_path,2]],dtype=int)
soma_diameter = distance_transform[soma_skeleton_coordinates[0,:],soma_skeleton_coordinates[1,:],soma_skeleton_coordinates[2,:]]

In [4]:
from neuron import h
import numpy as np
from matplotlib import pyplot

soma = h.Section(name='soma')
soma.nseg = 100
soma.Ra = 100
'''
soma.L = 1000
diam = np.random.randint(1,10,101)
for i,seg in enumerate(soma.allseg()):
    seg.diam = diam[i-1]
'''
h.pt3dclear(sec=soma)
xvec = h.Vector(soma_skeleton_coordinates[0,:])
yvec = h.Vector(soma_skeleton_coordinates[1,:])
zvec = h.Vector(soma_skeleton_coordinates[2,:])
dvec = h.Vector(soma_diameter)
h.pt3dadd(xvec, yvec, zvec, dvec, sec=soma)

soma.insert('hh')
for i,seg in enumerate(soma):
    seg.hh.gnabar = 0.12  # Sodium conductance in S/cm2
    seg.hh.gkbar = 0.036  # Potassium conductance in S/cm2
    seg.hh.gl = 0.0003    # Leak conductance in S/cm2
    seg.hh.el = -54.3     # Reversal potential in mV
#stim = h.IClamp(soma(0.5))
#stim.delay = 10
#stim.dur = 1
#stim.amp = 0.001
s = h.Shape()
s.show(False)

v_vec = h.Vector()        # Membrane potential vector
t_vec = h.Vector()        # Time stamp vector
v_vec.record(soma(0.5)._ref_v)
t_vec.record(h._ref_t)
simdur = 25.0
h.load_file('stdrun.hoc')
h.finitialize(-70)
h.continuerun(40)
h.topology()


|----------------------------------------------------------------------------------------------------|       soma(0-1)



1.0

In [5]:
distance_transform = ndi.distance_transform_edt(np.load('np arrays/foreground.npy'))
skeleton_image = np.load('np arrays/skeleton_image.npy') #dendrites
skeleton_obj = skan.Skeleton(skeleton_image)
pruning(skeleton_image,skeleton_obj,distance_transform)

In [11]:
#cross detection
nodes = np.zeros(len(skeleton_obj.coordinates))
paths = skeleton_obj.paths_list()
for i in range(len(paths)):
    nodes[paths[i][0]] = 1
    nodes[paths[i][-1]] = 1
nodes_index = np.where(nodes==1)
for id in nodes_index:
    x = skeleton_obj.coordinates[id,0]
    y = skeleton_obj.coordinates[id,1]
    z = skeleton_obj.coordinates[id,2]
    local_foreground_range = (distance_transform[x,y,z]*1.5)**2
    bifercations = []
    orientation = []
    inverse = []
    for i in len(paths):
        if(paths[i][0]==id):
            the_other_end = -1
            local_path = paths(i)
            inverse.append(False)
        elif(paths[i][-1]==id):
            the_other_end = 0
            local_path = paths(i)[::-1]
            inverse.append(True)
        else:
            continue
        
        bifercations.append(i)
        short_branch = True
        for coord in local_path:
            vx = skeleton_obj.coordinates[coord,0]-x
            vy = skeleton_obj.coordinates[coord,1]-y
            vz = skeleton_obj.coordinates[coord,2]-z
            if(vx**2+vy**2+vz**2>local_foreground_range):
                orientation.append(vx,vy,vz)
                short_branch = False
                break
        if(short_branch):
            orientation.append([skeleton_obj.coordinates[paths[i][the_other_end],0] - x,
            skeleton_obj.coordinates[paths[i][the_other_end],1] - y,
            skeleton_obj.coordinates[paths[i][the_other_end],2] - z])
    N_bifercation = len(bifercations)
    cross_products=np.zeros(N_bifercation,N_bifercation)-1
    for r in range(N_bifercation-1):
        for c in range(r+1,N_bifercation,1):
            cross_products[r,c]=np.abs(np.dot(orientation[c],orientation[r]))
    left_branch = 0
    if(len(bifercations)%2==1):
        left_branch=3
    while sum(cross_products>0)>left_branch:
        r,c = np.argmax(cross_products)
        cross_products[r,:] = -1
        cross_products[:,c] = -1
        if (inverse[r] and (not inverse[c])):
            paths(bifercations[r]).extend(paths(bifercations[c])[1:])
            paths.pop(bifercations[c])
        elif (inverse[r] and inverse[c]):
            paths(bifercations[r]).extend(paths(bifercations[c])[-2::-1])
            paths.pop(bifercations[c])
        elif ((not inverse[r]) and inverse[c]):
            paths(bifercations[c]).extend(paths(bifercations[r])[1:])
            paths.pop(bifercations[r])
        elif ((not inverse[r]) and (not inverse[c])):
            paths.append([*paths(bifercations[r])[-1:0:-1],*paths(bifercations[c])])
            paths.pop(bifercations[r])
            paths.pop(bifercations[c])
        else:
            print("error: path mismatch")
        


[  0.  21. 158.] [  0. 112.  41.] [  0. 177. 268.]


IndexError: arrays used as indices must be of integer (or boolean) type

In [ ]:
# net building
skeleton_node = np.zeros([3,np.shape(skeleton_image)[0],np.shape(skeleton_image)[1],np.shape(skeleton_image)[2]])
skeleton_node[0,:,:,:] = skeleton_image

summary=skan.summarize(skeleton_obj)
net = np.zeros([5,len(summary.index)])
net[0,:]=summary['skeleton-id']
root=[]
coordinates = np.array((skeleton_obj.coordinates),dtype=int)
skeleton_obj
for i in summary.index:
    if(summary['branch-type'][i]==1):
            root.append(i)
    bifercations=[]
    orientation=[]
    for j in summary.index[i+1:]:
        if(summary['node-id-dst'][i]==summary['node-id-src'][j]):
            bifercations.append(j)
            
            path = skeleton_obj.path(j)
            
        
        for b in bifercations:

            if(net[1,i]==0):
                net[1,i] = summary['skeleton-id'][j]
            elif(net[2,i]==0):
                net[2,i] = summary['skeleton-id'][j]
            else:
                skeleton_node[1,int(summary['image-coord-dst-0'][i]),int(summary['image-coord-dst-1'][i]),int(summary['image-coord-dst-2'][i])] = 1
                print("warning, more than 2 children attached to branch ",i)
                print(summary['image-coord-dst-0'][i],summary['image-coord-dst-1'][i],summary['image-coord-dst-2'][i])
            if(net[3,j]==0):
                net[3,j] = summary['skeleton-id'][i]
            elif(net[4,j]==0):
                net[4,j] = summary['skeleton-id'][i]
            else:
                skeleton_node[1,int(summary['image-coord-dst-0'][i]),int(summary['image-coord-dst-1'][i]),int(summary['image-coord-dst-2'][i])] = 1
                print("warning, more than 2 parents attached to branch ",j)


warning, more than 2 parents attached to branch  178
warning, more than 2 parents attached to branch  343
warning, more than 2 children attached to branch  386
46.0 408.0 262.0
warning, more than 2 parents attached to branch  514
warning, more than 2 children attached to branch  766
73.0 374.0 200.0
warning, more than 2 parents attached to branch  1477
warning, more than 2 parents attached to branch  1806
warning, more than 2 children attached to branch  1840
164.0 266.0 246.0
warning, more than 2 children attached to branch  1978
172.0 227.0 111.0
warning, more than 2 parents attached to branch  2184
warning, more than 2 children attached to branch  2207
179.0 253.0 220.0
warning, more than 2 parents attached to branch  2297
warning, more than 2 children attached to branch  2283
186.0 143.0 235.0
warning, more than 2 children attached to branch  2334
186.0 226.0 240.0
warning, more than 2 children attached to branch  2382
188.0 225.0 242.0
warning, more than 2 children attached to bra

In [4]:
print(np.shape(skeleton_node))
np.save('np arrays/skeleton_image_pruned', skeleton_image)
skeleton_node[2,:,:,:]=np.load('np arrays/foreground.npy')
imsave('images/skeleton_node.tif',np.einsum('ijkl->lijk', skeleton_node))

(3, 512, 512, 271)


In [13]:
soma_mask = ndi.binary_dilation(np.load('np arrays/soma_mask.npy'))
roots=[i for i in summary.index if (summary['branch-type'][i] == 1 and
			( soma_mask[int(summary['image-coord-dst-0'][i]),int(summary['image-coord-dst-1'][i]),int(summary['image-coord-dst-2'][i])]==1 or
            soma_mask[int(summary['image-coord-src-0'][i]),int(summary['image-coord-src-1'][i]),int(summary['image-coord-src-2'][i])]==1 )
            )]
